In [8]:
import cv2
from ultralytics import YOLO
import numpy as np

# تحميل نموذج YOLO
model = YOLO("yolov8n.pt")

# ألوان الفرق والحكم والكرة
colors = {
    "team1": (255, 0, 0),  # أزرق
    "team2": (0, 0, 255),  # أحمر
    "referee": (0, 255, 255),  # أصفر
    "ball": (0, 255, 0)  # أخضر
}

# قائمة الأحداث
events = []

def add_event(event):
    """إضافة حدث جديد."""
    events.append(event)
    if len(events) > 5:
        events.pop(0)

def draw_text(frame, text, position, color=(0, 255, 0)):
    """رسم النصوص على الفيديو."""
    cv2.putText(frame, text, position, cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)

def calculate_distance(p1, p2):
    """حساب المسافة بين نقطتين."""
    return np.linalg.norm(np.array(p1) - np.array(p2))

def process_frame(frame):
    """معالجة الإطار وتحليل الأحداث."""
    results = model(frame)
    players = []
    ball = None
    height, width, _ = frame.shape

    # تحليل النتائج
    for result in results:
        boxes = result.boxes.xyxy
        classes = result.boxes.cls
        
        for box, cls in zip(boxes, classes):
            x1, y1, x2, y2 = map(int, box)
            label = result.names[int(cls)]

            # كشف اللاعبين والكرة
            if label == "person":
                center = ((x1 + x2) // 2, (y1 + y2) // 2)
                height_ratio = (y2 - y1) / height

                # كشف السقوط: اللاعب الساقط غالباً يكون أقرب للأرض (y2 قريب من أسفل الشاشة)
                if height_ratio < 0.3:  # إذا كان اللاعب قصيرًا مقارنة بالباقين
                    add_event("لاعب سقط!")
                    draw_text(frame, "سقوط!", (x1, y1 - 10), (0, 0, 255))

                players.append((center[0], y2))  # حفظ موقع اللاعب (منتصف أسفل المربع)
                color = colors["team1"] if x1 < width // 2 else colors["team2"]
                cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
            elif label == "sports ball":
                ball = ((x1 + x2) // 2, (y1 + y2) // 2)
                cv2.circle(frame, ball, 10, colors["ball"], -1)

    # كشف التسلل وتمريرات التسلل
    if len(players) > 1 and ball:
        players.sort(key=lambda p: p[0])  # ترتيب اللاعبين حسب الموقع الأفقي
        last_defender = players[-2]  # آخر مدافع (قبل آخر لاعب لأنه غالبًا الحارس)

        # رسم خط التسلل
        cv2.line(frame, (last_defender[0], 0), (last_defender[0], height), (0, 255, 255), 2)

        # تمريرة مشبوهة: إذا كان اللاعب المتقدم خلف آخر مدافع لحظة تمرير الكرة
        for player in players:
            if player[0] > last_defender[0] and player != players[-1]:
                add_event("تمرير مشبوه!")
                draw_text(frame, "تمرير مشبوه!", (player[0] - 40, player[1] - 10), (0, 0, 255))
                break

    # حساب المسافات وعرضها
    if ball:
        for player in players:
            distance = calculate_distance(player, ball)
            draw_text(frame, f"{int(distance)} px", (player[0] + 10, player[1] - 10), (0, 255, 255))

    # عرض الأحداث
    y_offset = 50
    for event in events:
        draw_text(frame, event, (50, y_offset), (0, 0, 255))
        y_offset += 30

    return frame

# فتح الفيديو أو الكاميرا
cap = cv2.VideoCapture("match_video.mp4")  # غيّرها لـ 0 إذا أردت استخدام الكاميرا مباشرة

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # معالجة الإطار وعرضه
    processed_frame = process_frame(frame)
    cv2.imshow("VAR System", processed_frame)

    # الخروج بالضغط على "q"
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


0: 384x640 7 persons, 420.1ms
Speed: 123.9ms preprocess, 420.1ms inference, 51.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 persons, 254.2ms
Speed: 30.9ms preprocess, 254.2ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 persons, 211.6ms
Speed: 5.6ms preprocess, 211.6ms inference, 2.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 9 persons, 179.9ms
Speed: 5.0ms preprocess, 179.9ms inference, 2.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 persons, 1 sports ball, 186.1ms
Speed: 8.4ms preprocess, 186.1ms inference, 1.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 persons, 1 sports ball, 202.9ms
Speed: 10.3ms preprocess, 202.9ms inference, 2.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 persons, 1 sports ball, 206.1ms
Speed: 7.4ms preprocess, 206.1ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 persons, 1 sports ball, 176.7ms
S

In [5]:
import cv2
from ultralytics import YOLO
import numpy as np

# تحميل نموذج YOLO
model = YOLO("yolov8n.pt")

# ألوان الفرق والحكم والكرة
colors = {
    "team1": (255, 0, 0),  # أزرق
    "team2": (0, 0, 255),  # أحمر
    "referee": (0, 255, 255),  # أصفر
    "ball": (0, 255, 0)  # أخضر
}

# قائمة الأحداث
events = []

def add_event(event):
    """إضافة حدث جديد."""
    events.append(event)
    if len(events) > 5:
        events.pop(0)

def draw_text(frame, text, position, color=(0, 255, 0)):
    """رسم النصوص على الفيديو."""
    cv2.putText(frame, text, position, cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)

def calculate_distance(p1, p2):
    """حساب المسافة بين نقطتين."""
    return np.linalg.norm(np.array(p1) - np.array(p2))

def process_frame(frame):
    """معالجة الإطار وتحليل الأحداث."""
    results = model(frame)
    players = []
    ball = None
    height, width, _ = frame.shape

    # تحليل النتائج
    for result in results:
        boxes = result.boxes.xyxy
        classes = result.boxes.cls
        
        for box, cls in zip(boxes, classes):
            x1, y1, x2, y2 = map(int, box)
            label = result.names[int(cls)]

            # كشف اللاعبين والكرة
            if label == "person":
                center = ((x1 + x2) // 2, (y1 + y2) // 2)
                height_ratio = (y2 - y1) / height

                # كشف السقوط: اللاعب الساقط غالباً يكون أقرب للأرض (y2 قريب من أسفل الشاشة)
                if height_ratio < 0.3:  # إذا كان اللاعب قصيرًا مقارنة بالباقين
                    add_event("لاعب سقط!")
                    draw_text(frame, "سقوط!", (x1, y1 - 10), (0, 0, 255))

                players.append((center[0], y2))  # حفظ موقع اللاعب (منتصف أسفل المربع)
                color = colors["team1"] if x1 < width // 2 else colors["team2"]
                cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
            elif label == "sports ball":
                ball = ((x1 + x2) // 2, (y1 + y2) // 2)
                cv2.circle(frame, ball, 10, colors["ball"], -1)

    # كشف التسلل وتمريرات التسلل
    if len(players) > 1 and ball:
        players.sort(key=lambda p: p[0])  # ترتيب اللاعبين حسب الموقع الأفقي
        last_defender = players[-2]  # آخر مدافع (قبل آخر لاعب لأنه غالبًا الحارس)

        # رسم خط التسلل
        cv2.line(frame, (last_defender[0], 0), (last_defender[0], height), (0, 255, 255), 2)

        # تمريرة مشبوهة: إذا كان اللاعب المتقدم خلف آخر مدافع لحظة تمرير الكرة
        for player in players:
            if player[0] > last_defender[0] and player != players[-1]:
                add_event("تمرير مشبوه!")
                draw_text(frame, "تمرير مشبوه!", (player[0] - 40, player[1] - 10), (0, 0, 255))
                break

    # حساب المسافات وعرضها
    if ball:
        for player in players:
            distance = calculate_distance(player, ball)
            draw_text(frame, f"{int(distance)} px", (player[0] + 10, player[1] - 10), (0, 255, 255))

    # عرض الأحداث
    y_offset = 50
    for event in events:
        draw_text(frame, event, (50, y_offset), (0, 0, 255))
        y_offset += 30

    return frame

# فتح الفيديو أو الكاميرا
cap = cv2.VideoCapture("match_video.mp4")  # غيّرها لـ 0 إذا أردت استخدام الكاميرا مباشرة

# إعداد الكتابة في ملف فيديو جديد
fourcc = cv2.VideoWriter_fourcc(*'XVID')  # استخدم XVID لضغط الفيديو
out = cv2.VideoWriter('output3.avi', fourcc, 20.0, (640, 480))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # معالجة الإطار وعرضه
    processed_frame = process_frame(frame)
    
    # كتابة الإطار المعالج في ملف الفيديو
    out.write(processed_frame)

    # الخروج بالضغط على "q"
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
out.release()
cv2.destroyAllWindows()

In [9]:
import cv2
from ultralytics import YOLO
import numpy as np

# تحميل نموذج YOLO
model = YOLO("yolov8n.pt")

# ألوان الفرق والحكم والكرة
colors = {
    "team1": (255, 0, 0),  # أزرق
    "team2": (0, 0, 255),  # أحمر
    "referee": (0, 255, 255),  # أصفر
    "ball": (0, 255, 0)  # أخضر
}

# قائمة الأحداث
events = []

def add_event(event):
    """إضافة حدث جديد وحفظه في ملف نصي."""
    events.append(event)
    # كتابة الحدث في ملف نصي
    with open('events_log.txt', 'a') as f:
        f.write(event + '\n')  # كتابة الحدث في سطر جديد
    if len(events) > 5:
        events.pop(0)

def draw_text(frame, text, position, color=(0, 255, 0)):
    """رسم النصوص على الفيديو."""
    cv2.putText(frame, text, position, cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)

def calculate_distance(p1, p2):
    """حساب المسافة بين نقطتين."""
    return np.linalg.norm(np.array(p1) - np.array(p2))

def process_frame(frame):
    """معالجة الإطار وتحليل الأحداث."""
    results = model(frame)
    players = []
    ball = None
    height, width, _ = frame.shape

    # تحليل النتائج
    for result in results:
        boxes = result.boxes.xyxy
        classes = result.boxes.cls
        
        for box, cls in zip(boxes, classes):
            x1, y1, x2, y2 = map(int, box)
            label = result.names[int(cls)]

            # كشف اللاعبين والكرة
            if label == "person":
                center = ((x1 + x2) // 2, (y1 + y2) // 2)
                height_ratio = (y2 - y1) / height

                # كشف السقوط: اللاعب الساقط غالباً يكون أقرب للأرض (y2 قريب من أسفل الشاشة)
                if height_ratio < 0.3:  # إذا كان اللاعب قصيرًا مقارنة بالباقين
                    add_event("لاعب سقط!")
                    draw_text(frame, "سقوط!", (x1, y1 - 10), (0, 0, 255))

                players.append((center[0], y2))  # حفظ موقع اللاعب (منتصف أسفل المربع)
                color = colors["team1"] if x1 < width // 2 else colors["team2"]
                cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
            elif label == "sports ball":
                ball = ((x1 + x2) // 2, (y1 + y2) // 2)
                cv2.circle(frame, ball, 10, colors["ball"], -1)

    # كشف التسلل وتمريرات التسلل
    if len(players) > 1 and ball:
        players.sort(key=lambda p: p[0])  # ترتيب اللاعبين حسب الموقع الأفقي
        last_defender = players[-2]  # آخر مدافع (قبل آخر لاعب لأنه غالبًا الحارس)

        # رسم خط التسلل
        cv2.line(frame, (last_defender[0], 0), (last_defender[0], height), (0, 255, 255), 2)

        # تمريرة مشبوهة: إذا كان اللاعب المتقدم خلف آخر مدافع لحظة تمرير الكرة
        for player in players:
            if player[0] > last_defender[0] and player != players[-1]:
                add_event("تمرير مشبوه!")
                draw_text(frame, "تمرير مشبوه!", (player[0] - 40, player[1] - 10), (0, 0, 255))
                break

    # حساب المسافات وعرضها
    if ball:
        for player in players:
            distance = calculate_distance(player, ball)
            draw_text(frame, f"{int(distance)} px", (player[0] + 10, player[1] - 10), (0, 255, 255))

    # عرض الأحداث
    y_offset = 50
    for event in events:
        draw_text(frame, event, (50, y_offset), (0, 0, 255))
        y_offset += 30

    return frame

# فتح الفيديو أو الكاميرا
cap = cv2.VideoCapture("match_video.mp4")  # غيّرها لـ 0 إذا أردت استخدام الكاميرا مباشرة

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # معالجة الإطار وعرضه
    processed_frame = process_frame(frame)
    cv2.imshow("VAR System", processed_frame)

    # الخروج بالضغط على "q"
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()



0: 384x640 7 persons, 182.3ms
Speed: 8.5ms preprocess, 182.3ms inference, 2.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 persons, 309.8ms
Speed: 39.5ms preprocess, 309.8ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 persons, 239.4ms
Speed: 43.3ms preprocess, 239.4ms inference, 2.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 9 persons, 234.4ms
Speed: 15.5ms preprocess, 234.4ms inference, 1.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 persons, 1 sports ball, 332.0ms
Speed: 110.5ms preprocess, 332.0ms inference, 2.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 persons, 1 sports ball, 225.0ms
Speed: 14.6ms preprocess, 225.0ms inference, 2.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 persons, 1 sports ball, 212.5ms
Speed: 7.4ms preprocess, 212.5ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 persons, 1 sports ball, 195.6ms


In [ ]:
import cv2
from ultralytics import YOLO
import numpy as np
import os
import time

# تحميل نموذج YOLO
model = YOLO("yolov8n.pt")

# ألوان الفرق والحكم والكرة
colors = {
    "team1": (255, 0, 0),  # أزرق
    "team2": (0, 0, 255),  # أحمر
    "referee": (0, 255, 255),  # أصفر
    "ball": (0, 255, 0)  # أخضر
}

# قائمة الأحداث
events = []

def clear_screen():
    """مسح شاشة موجه الأوامر."""
    os.system('cls' if os.name == 'nt' else 'clear')

def add_event(event):
    """إضافة حدث جديد وطباعة الأحداث في موجه الأوامر."""
    events.append(event)
    if len(events) > 5:
        events.pop(0)
    clear_screen()  # مسح الشاشة قبل طباعة الأحداث الجديدة
    print("الأحداث الحالية:")
    for e in events:
        print(f"- {e}")
    print("\n")  # إضافة سطر فارغ للتوضيح

def draw_text(frame, text, position, color=(0, 255, 0)):
    """رسم النصوص على الفيديو."""
    cv2.putText(frame, text, position, cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)

def calculate_distance(p1, p2):
    """حساب المسافة بين نقطتين."""
    return np.linalg.norm(np.array(p1) - np.array(p2))

def process_frame(frame):
    """معالجة الإطار وتحليل الأحداث."""
    results = model(frame)
    players = []
    ball = None
    height, width, _ = frame.shape

    # تحليل النتائج
    for result in results:
        boxes = result.boxes.xyxy
        classes = result.boxes.cls
        
        for box, cls in zip(boxes, classes):
            x1, y1, x2, y2 = map(int, box)
            label = result.names[int(cls)]

            # كشف اللاعبين والكرة
            if label == "person":
                center = ((x1 + x2) // 2, (y1 + y2) // 2)
                height_ratio = (y2 - y1) / height

                # كشف السقوط: اللاعب الساقط غالباً يكون أقرب للأرض (y2 قريب من أسفل الشاشة)
                if height_ratio < 0.3:  # إذا كان اللاعب قصيرًا مقارنة بالباقين
                    add_event("لاعب سقط!")
                    draw_text(frame, "سقوط!", (x1, y1 - 10), (0, 0, 255))

                players.append((center[0], y2))  # حفظ موقع اللاعب (منتصف أسفل المربع)
                color = colors["team1"] if x1 < width // 2 else colors["team2"]
                cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
            elif label == "sports ball":
                ball = ((x1 + x2) // 2, (y1 + y2) // 2)
                cv2.circle(frame, ball, 10, colors["ball"], -1)

    # كشف التسلل وتمريرات التسلل
    if len(players) > 1 and ball:
        players.sort(key=lambda p: p[0])  # ترتيب اللاعبين حسب الموقع الأفقي
        last_defender = players[-2]  # آخر مدافع (قبل آخر لاعب لأنه غالبًا الحارس)

        # رسم خط التسلل
        cv2.line(frame, (last_defender[0], 0), (last_defender[0], height), (0, 255, 255), 2)

        # تمريرة مشبوهة: إذا كان اللاعب المتقدم خلف آخر مدافع لحظة تمرير الكرة
        for player in players:
            if player[0] > last_defender[0] and player != players[-1]:
                add_event("تمرير مشبوه!")
                draw_text(frame, "تمرير مشبوه!", (player[0] - 40, player[1] - 10), (0, 0, 255))
                break

    # حساب المسافات وعرضها
    if ball:
        for player in players:
            distance = calculate_distance(player, ball)
            draw_text(frame, f"{int(distance)} px", (player[0] + 10, player[1] - 10), (0, 255, 255))

    return frame

# فتح الفيديو أو الكاميرا
cap = cv2.VideoCapture("match_video.mp4")  # غيّرها لـ 0 إذا أردت استخدام الكاميرا مباشرة

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # معالجة الإطار وعرضه
    processed_frame = process_frame(frame)
    cv2.imshow("VAR System", processed_frame)

    # الخروج بالضغط على "q"
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


0: 384x640 7 persons, 298.9ms
Speed: 11.6ms preprocess, 298.9ms inference, 2.5ms postprocess per image at shape (1, 3, 384, 640)
الأحداث الحالية:
- لاعب سقط!


الأحداث الحالية:
- لاعب سقط!
- لاعب سقط!


الأحداث الحالية:
- لاعب سقط!
- لاعب سقط!
- لاعب سقط!


الأحداث الحالية:
- لاعب سقط!
- لاعب سقط!
- لاعب سقط!
- لاعب سقط!


الأحداث الحالية:
- لاعب سقط!
- لاعب سقط!
- لاعب سقط!
- لاعب سقط!
- لاعب سقط!


الأحداث الحالية:
- لاعب سقط!
- لاعب سقط!
- لاعب سقط!
- لاعب سقط!
- لاعب سقط!


الأحداث الحالية:
- لاعب سقط!
- لاعب سقط!
- لاعب سقط!
- لاعب سقط!
- لاعب سقط!



0: 384x640 6 persons, 197.7ms
Speed: 5.8ms preprocess, 197.7ms inference, 2.6ms postprocess per image at shape (1, 3, 384, 640)
الأحداث الحالية:
- لاعب سقط!
- لاعب سقط!
- لاعب سقط!
- لاعب سقط!
- لاعب سقط!


الأحداث الحالية:
- لاعب سقط!
- لاعب سقط!
- لاعب سقط!
- لاعب سقط!
- لاعب سقط!


الأحداث الحالية:
- لاعب سقط!
- لاعب سقط!
- لاعب سقط!
- لاعب سقط!
- لاعب سقط!


الأحداث الحالية:
- لاعب سقط!
- لاعب سقط!
- لاعب سقط!
- لاعب سقط!
- لاعب

In [10]:
import cv2
import numpy as np
from ultralytics import YOLO

# تحميل نموذج YOLOv8 المدرب على كشف الأشخاص والكرة
model = YOLO("yolov8n.pt")

# فتح الفيديو
video_path = "match_video.mp4"
cap = cv2.VideoCapture(video_path)

# إعداد الألوان
team1_color = (0, 255, 0)  # أخضر
team2_color = (255, 0, 0)  # أزرق
referee_color = (0, 0, 255)  # أحمر
ball_color = (0, 255, 255)  # أصفر
card_color = (0, 0, 255)  # لون للتحذير عند ظهور بطاقة حمراء أو صفراء

# تحديد الفرق بناءً على مواقع اللاعبين (يفضل استخدام AI إضافي هنا)
team1_side = "left"  # فريق 1 على اليسار
team2_side = "right"  # فريق 2 على اليمين

def detect_offside(players_positions):
    """
    تحليل مواقع اللاعبين لتحديد التسلل.
    """
    players_positions.sort()
    if len(players_positions) > 2:
        offside_line = players_positions[1]  # ثاني آخر مدافع هو خط التسلل
        return offside_line
    return None

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # تشغيل YOLO على الإطار الحالي
    results = model(frame)

    player_positions = []
    detected_objects = []

    for result in results:
        for box in result.boxes.xyxy:
            x1, y1, x2, y2 = map(int, box[:4])  # إحداثيات الصندوق
            label = result.names[int(box[5])]   # اسم الكائن المكتشف
            
            if label == "person":  
                # تقدير لون القميص لتحديد الفريق (يفضل AI متقدم هنا)
                if x1 < frame.shape[1] // 2:
                    team_color = team1_color
                    team_label = "player_team1"
                else:
                    team_color = team2_color
                    team_label = "player_team2"
                
                player_positions.append(x1)
                detected_objects.append((x1, y1, x2-x1, y2-y1, team_label))
                cv2.rectangle(frame, (x1, y1), (x2, y2), team_color, 2)
            
            elif label == "sports ball":
                cv2.circle(frame, ((x1 + x2) // 2, (y1 + y2) // 2), 10, ball_color, -1)
            
            elif label == "referee":
                cv2.rectangle(frame, (x1, y1), (x2, y2), referee_color, 2)

    # تحليل التسلل
    offside_line = detect_offside(player_positions)
    if offside_line:
        cv2.line(frame, (offside_line, 0), (offside_line, frame.shape[0]), (0, 0, 255), 2)
        cv2.putText(frame, "OFFSIDE!", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    # عرض النتيجة
    cv2.imshow("VAR Analysis", frame)
    if cv2.waitKey(25) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()



0: 384x640 7 persons, 269.5ms
Speed: 14.0ms preprocess, 269.5ms inference, 2.6ms postprocess per image at shape (1, 3, 384, 640)


IndexError: index 5 is out of bounds for dimension 0 with size 4

In [5]:
import cv2
import numpy as np
from ultralytics import YOLO

# تحميل نموذج YOLOv8
model = YOLO("yolov8n.pt")

# فتح الفيديو
video_path = "match_video.mp4"
cap = cv2.VideoCapture(video_path)

# إعداد الألوان
team1_color = (0, 255, 0)  # أخضر
team2_color = (255, 0, 0)  # أزرق
referee_color = (0, 0, 255)  # أحمر
ball_color = (0, 255, 255)  # أصفر

# نتيجة المباراة
score_team1 = 0
score_team2 = 0

# مواقع المرمى (يجب تعديلها وفقًا للفيديو)
goal_left_x = 50  # موقع مرمى الفريق 1
goal_right_x = 1200  # موقع مرمى الفريق 2

def detect_goal(ball_x):
    """
    التحقق مما إذا كانت الكرة داخل المرمى.
    """
    global score_team1, score_team2
    
    if ball_x < goal_left_x:
        score_team2 += 1  # هدف للفريق 2
        return "Goal for Team 2!"
    elif ball_x > goal_right_x:
        score_team1 += 1  # هدف للفريق 1
        return "Goal for Team 1!"
    return None

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    results = model(frame)

    goal_text = None
    ball_x = None

    for result in results:
        for box in result.boxes.xyxy:
            x1, y1, x2, y2 = map(int, box[:4])
            label = result.names[int(box[5])]

            if label == "sports ball":
                ball_x = (x1 + x2) // 2
                cv2.circle(frame, (ball_x, (y1 + y2) // 2), 10, ball_color, -1)

    # التحقق من تسجيل الأهداف
    if ball_x is not None:
        goal_text = detect_goal(ball_x)

    # عرض النتيجة
    score_text = f"Team 1: {score_team1}  |  Team 2: {score_team2}"
    cv2.putText(frame, score_text, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

    if goal_text:
        cv2.putText(frame, goal_text, (500, 100), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 255, 255), 3)

    cv2.imshow("VAR Analysis", frame)
    if cv2.waitKey(25) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [7]:
import cv2
import numpy as np

# تحميل نموذج للكشف عن اللاعبين (مثل YOLO أو Faster R-CNN)
# هنا نفترض أنك قد قمت بتحميل نموذج جاهز للكشف عن الأشخاص
# ونماذج التسلل والبطاقات الصفراء يجب أن يتم تدريبها أو تحديدها يدويًا في سياق الفيديو

def analyze_frame(frame):
    # تحليل الإطار للبحث عن التسلل أو الكرت الأصفر
    # هنا نضع منطقك للكشف عن التسلل أو الكرت الأصفر
    # على سبيل المثال: إذا كان لاعب في موقف تسلل
    # أو إذا تم اكتشاف سلوكيات تشير إلى بطاقة صفراء
    event_detected = None
    
    # مثلاً فرضًا تم اكتشاف تسلل أو بطاقة صفراء
    # ملاحظة: هذا مجرد مثال بسيط ويجب تطويره أكثر بناءً على النموذج المدرب أو المنطق

    return event_detected

# تحميل الفيديو
cap = cv2.VideoCapture('match_video.mp4')

# إعداد الكتابة للفيديو الناتج
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('output_video.avi', fourcc, 20.0, (640, 480))

while(cap.isOpened()):
    ret, frame = cap.read()
    if not ret:
        break

    # تحليل الإطار للكشف عن الأحداث
    event = analyze_frame(frame)

    if event == 'yellow_card':
        cv2.putText(frame, 'Yellow Card!', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2, cv2.LINE_AA)
    elif event == 'offside':
        cv2.putText(frame, 'Offside!', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)

    # كتابة الإطار مع النص إلى الفيديو النهائي
    out.write(frame)

    # عرض الإطار (يمكن إيقافه في حالة عدم الرغبة في مشاهدة المعالجة في الوقت الحقيقي)
    cv2.imshow('Frame', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# إغلاق جميع النوافذ والملفات
cap.release()
out.release()
cv2.destroyAllWindows()

In [1]:
import cv2
import mediapipe as mp
import numpy as np

# تحميل نموذج كشف الجسم من MediaPipe
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

# تحميل الفيديو
video_path = "tennis_match.mp4"  # استبدل باسم ملف الفيديو لديك
cap = cv2.VideoCapture(video_path)

# متغيرات لتتبع الحركة
previous_ball_position = None
ball_speeds = []
player_positions = []

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # تحويل الإطار إلى RGB
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # تحليل الإطار باستخدام MediaPipe لاكتشاف اللاعبين
    results = pose.process(frame_rgb)

    # رسم النقاط الهيكلية للاعبين
    if results.pose_landmarks:
        for landmark in results.pose_landmarks.landmark:
            x = int(landmark.x * frame.shape[1])
            y = int(landmark.y * frame.shape[0])
            cv2.circle(frame, (x, y), 5, (0, 255, 0), -1)
        player_positions.append((x, y))

    # تحويل الصورة إلى تدرج رمادي للبحث عن الكرة
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    circles = cv2.HoughCircles(blurred, cv2.HOUGH_GRADIENT, 1, 20,
                               param1=50, param2=30, minRadius=5, maxRadius=20)

    # إذا تم العثور على كرة
    if circles is not None:
        circles = np.uint16(np.around(circles))
        for circle in circles[0, :]:
            x, y, r = circle
            cv2.circle(frame, (x, y), r, (0, 0, 255), 2)

            # حساب سرعة الكرة
            if previous_ball_position is not None:
                dx = x - previous_ball_position[0]
                dy = y - previous_ball_position[1]
                speed = np.sqrt(dx**2 + dy**2)
                ball_speeds.append(speed)

            previous_ball_position = (x, y)

    # عرض الفيديو مع التحليل
    cv2.imshow("Tennis Analysis", frame)

    if cv2.waitKey(25) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()

# طباعة بعض الإحصائيات
if ball_speeds:
    print(f"متوسط سرعة الكرة: {np.mean(ball_speeds):.2f} بكسل/إطار")

ImportError: DLL load failed while importing _framework_bindings: A dynamic link library (DLL) initialization routine failed.

In [2]:
import cv2
import numpy as np
import pandas as pd
from ultralytics import YOLO

# تحميل نموذج YOLO
model = YOLO("yolov8n.pt")  

# فتح الفيديو
video_path = "match2.mp4"
cap = cv2.VideoCapture(video_path)

# إنشاء DataFrame لتسجيل الحالات التحكيمية
columns = ["Frame", "Event", "Confidence"]
events_log = pd.DataFrame(columns=columns)

frame_count = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_count += 1  # عداد الإطارات

    # تحليل الإطار
    results = model(frame)

    for r in results:
        for box in r.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            conf = box.conf[0].item()
            cls = int(box.cls[0].item())

            # كشف البطاقات الصفراء والحمراء (افتراضيًا الحكم يحمل كرتًا)
            if cls == 1:  # الحكم
                if conf > 0.6:  # فقط الحالات ذات الثقة العالية
                    event = "Referee Detected"
                    events_log.loc[len(events_log)] = [frame_count, event, conf]

            # كشف التسلل (كمثال: لاعب في موقع مشبوه)
            elif cls == 0:  # اللاعب
                if conf > 0.7:  # ثقة عالية
                    event = "Potential Offside"
                    events_log.loc[len(events_log)] = [frame_count, event, conf]

            # كشف الأخطاء (كمثال: سقوط لاعب)
            elif cls == 2:  # لاعب ساقط (يجب تدريب النموذج على ذلك)
                event = "Foul Detected"
                events_log.loc[len(events_log)] = [frame_count, event, conf]

            # رسم الصندوق حول الأهداف المكتشفة
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, f"{event} {conf:.2f}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # عرض الفيديو
    cv2.imshow("Match Analysis", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

# حفظ التقرير في CSV بعد انتهاء التحليل
events_log.to_csv("match_analysis.csv", index=False)
print("\n📊 **التقرير النهائي**")
print(events_log)

# عرض ملخص التحليل
summary = events_log["Event"].value_counts()
print("\n🔍 **إحصائيات التحكيم**")
print(summary)


0: 384x640 17 persons, 341.5ms
Speed: 28.6ms preprocess, 341.5ms inference, 25.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 16 persons, 505.6ms
Speed: 14.5ms preprocess, 505.6ms inference, 2.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 15 persons, 291.9ms
Speed: 7.2ms preprocess, 291.9ms inference, 4.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 15 persons, 235.3ms
Speed: 5.5ms preprocess, 235.3ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 19 persons, 202.8ms
Speed: 5.9ms preprocess, 202.8ms inference, 3.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 17 persons, 222.5ms
Speed: 7.1ms preprocess, 222.5ms inference, 2.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 17 persons, 182.4ms
Speed: 4.3ms preprocess, 182.4ms inference, 2.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 15 persons, 216.2ms
Speed: 7.4ms preprocess, 216.2ms inference, 2.5ms postp

In [3]:
import cv2
import numpy as np
import pandas as pd
from ultralytics import YOLO

# تحميل نموذج YOLO المدرب مسبقًا
model = YOLO("yolov8n.pt")  # استخدم نموذجًا مخصصًا للكرة واللاعبين إن وجد

# فتح الفيديو
video_path = "match2.mp4"
cap = cv2.VideoCapture(video_path)

# الحصول على خصائص الفيديو
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# إعداد فيديو الإخراج
output_path = "match_analysis.mp4"
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # صيغة MP4
out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

# إنشاء DataFrame لتسجيل الأحداث
columns = ["Frame", "Event", "Confidence"]
events_log = pd.DataFrame(columns=columns)

frame_count = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_count += 1  # عداد الإطارات

    # تحليل الإطار باستخدام YOLO
    results = model(frame)

    event_text = ""  # نص الحدث لعرضه في الفيديو

    for r in results:
        for box in r.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            conf = box.conf[0].item()
            cls = int(box.cls[0].item())

            # تصنيف الكائنات المكتشفة
            if cls == 0:  # اللاعب
                label = "Player"
                color = (0, 255, 0)  # أخضر
            elif cls == 1:  # الحكم
                label = "Referee"
                color = (255, 0, 0)  # أزرق
            elif cls == 2:  # الكرة
                label = "Ball"
                color = (0, 0, 255)  # أحمر
            else:
                label = f"Class {cls}"
                color = (255, 255, 0)  # أصفر

            # رسم الصندوق حول الهدف المكتشف
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
            cv2.putText(frame, f"{label} {conf:.2f}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

            # تحديد الحالات التحكيمية
            if cls == 1 and conf > 0.6:  # الحكم يحمل كرت
                event_text = "Referee Action Detected"
                events_log.loc[len(events_log)] = [frame_count, event_text, conf]

            elif cls == 0 and conf > 0.7:  # تسلل محتمل
                event_text = "Potential Offside"
                events_log.loc[len(events_log)] = [frame_count, event_text, conf]

            elif cls == 2 and conf > 0.7:  # الكرة مكتشفة (يمكن تحليل حركتها)
                event_text = "Ball in Play"

    # عرض الحدث في أعلى الشاشة
    if event_text:
        cv2.rectangle(frame, (0, 0), (frame_width, 50), (0, 0, 0), -1)  # شريط أسود أعلى الفيديو
        cv2.putText(frame, event_text, (20, 35), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

    # حفظ الإطار في الفيديو الجديد
    out.write(frame)

    # عرض الفيديو أثناء التشغيل
    cv2.imshow("Match Analysis", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
out.release()
cv2.destroyAllWindows()

# حفظ التقرير في CSV
events_log.to_csv("match_analysis2.csv", index=False)
print("\n📊 **التقرير النهائي**")
print(events_log)

# عرض إحصائيات التحليل
summary = events_log["Event"].value_counts()
print("\n🔍 **إحصائيات التحكيم**")
print(summary)

print(f"\n✅ تم حفظ الفيديو التحليلي: {output_path2}")


0: 384x640 17 persons, 376.0ms
Speed: 76.7ms preprocess, 376.0ms inference, 54.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 16 persons, 399.6ms
Speed: 9.5ms preprocess, 399.6ms inference, 14.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 15 persons, 217.4ms
Speed: 7.6ms preprocess, 217.4ms inference, 2.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 15 persons, 194.4ms
Speed: 5.1ms preprocess, 194.4ms inference, 2.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 19 persons, 206.2ms
Speed: 5.3ms preprocess, 206.2ms inference, 2.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 17 persons, 192.8ms
Speed: 4.4ms preprocess, 192.8ms inference, 2.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 17 persons, 181.2ms
Speed: 24.9ms preprocess, 181.2ms inference, 2.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 15 persons, 181.3ms
Speed: 7.7ms preprocess, 181.3ms inference, 1.3ms post

NameError: name 'output_path2' is not defined

In [5]:
import cv2
import numpy as np
import pandas as pd
from ultralytics import YOLO

# تحميل نموذج YOLO المدرب مسبقًا
model = YOLO("yolov8n.pt")  # استخدم نموذجًا مخصصًا إن توفر

# تحميل صورة الكرت الأصفر
yellow_card = cv2.imread("yellow_card.png", cv2.IMREAD_UNCHANGED)
yellow_card = cv2.imread("yellow_card.jpg", cv2.IMREAD_UNCHANGED)

# فتح الفيديو
video_path = "match2.mp4"
cap = cv2.VideoCapture(video_path)

# الحصول على خصائص الفيديو
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# إعداد فيديو الإخراج
output_path = "match_analysis3.mp4"
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # صيغة MP4
out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

# إنشاء DataFrame لتسجيل الحالات التحكيمية
columns = ["Frame", "Event", "Confidence"]
events_log = pd.DataFrame(columns=columns)

frame_count = 0
event_active = False  # للتحكم في ظهور الحدث
event_duration = 50   # عدد الإطارات التي يستمر فيها الحدث
event_counter = 0     # عداد الإطارات النشطة للحدث

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_count += 1  # عداد الإطارات

    # تحليل الإطار باستخدام YOLO
    results = model(frame)

    event_text = ""  # نص الحدث لعرضه في الفيديو
    show_yellow_card = False  # للتحكم في إظهار الكرت الأصفر

    for r in results:
        for box in r.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            conf = box.conf[0].item()
            cls = int(box.cls[0].item())

            # تصنيف الكائنات المكتشفة
            if cls == 0:  # اللاعب
                label = "Player"
                color = (0, 255, 0)  # أخضر
            elif cls == 1:  # الحكم
                label = "Referee"
                color = (255, 0, 0)  # أزرق
            elif cls == 2:  # الكرة
                label = "Ball"
                color = (0, 0, 255)  # أحمر
            else:
                label = f"Class {cls}"
                color = (255, 255, 0)  # أصفر

            # رسم الصندوق حول الهدف المكتشف
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
            cv2.putText(frame, f"{label} {conf:.2f}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

            # كشف حالة رفع الحكم للكرت الأصفر
            if cls == 1 and conf > 0.7:  # الحكم يحمل كرتًا
                event_text = "Yellow Card!"
                show_yellow_card = True
                events_log.loc[len(events_log)] = [frame_count, event_text, conf]
                event_active = True  # تفعيل الحدث
                event_counter = event_duration  # ضبط المؤقت

    # التحكم في عرض الحدث فقط عند وقوعه
    if event_active and event_counter > 0:
        # رسم شريط الحدث
        cv2.rectangle(frame, (0, 0), (frame_width, 50), (0, 0, 0), -1)  
        cv2.putText(frame, event_text, (20, 35), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

        # إدراج صورة الكرت الأصفر في أعلى الزاوية اليمنى
        if show_yellow_card:
            card_h, card_w, _ = yellow_card.shape
            frame[10:10+card_h, frame_width-10-card_w:frame_width-10] = yellow_card

        event_counter -= 1  # تقليل العداد
    else:
        event_active = False  # إيقاف الحدث بعد انتهاء المدة

    # حفظ الإطار في الفيديو الجديد
    out.write(frame)

    # عرض الفيديو أثناء التشغيل
    cv2.imshow("Match Analysis", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
out.release()
cv2.destroyAllWindows()

# حفظ التقرير في CSV
events_log.to_csv("match_analysis3.csv", index=False)
print("\n📊 **التقرير النهائي**")
print(events_log)

# عرض إحصائيات التحليل
summary = events_log["Event"].value_counts()
print("\n🔍 **إحصائيات التحكيم**")
print(summary)

print(f"\n✅ تم حفظ الفيديو التحليلي: {output_path3}")


0: 384x640 17 persons, 209.8ms
Speed: 34.6ms preprocess, 209.8ms inference, 8.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 16 persons, 202.3ms
Speed: 14.3ms preprocess, 202.3ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 15 persons, 345.8ms
Speed: 17.0ms preprocess, 345.8ms inference, 1.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 15 persons, 165.6ms
Speed: 3.9ms preprocess, 165.6ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 19 persons, 190.1ms
Speed: 7.3ms preprocess, 190.1ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 17 persons, 169.8ms
Speed: 7.5ms preprocess, 169.8ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 17 persons, 179.0ms
Speed: 3.8ms preprocess, 179.0ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 15 persons, 146.5ms
Speed: 3.2ms preprocess, 146.5ms inference, 3.1ms postp

NameError: name 'output_path3' is not defined

In [ ]:
import cv2
import numpy as np
import pandas as pd
from ultralytics import YOLO

# تحميل نموذج YOLO المدرب مسبقًا
model = YOLO("yolov8n.pt")  # استخدم نموذجًا مخصصًا إن توفر

# تحميل صورة الكرت الأصفر
yellow_card = cv2.imread("yellow_card.png", cv2.IMREAD_UNCHANGED)
yellow_card = cv2.imread("yellow_card.jpg", cv2.IMREAD_UNCHANGED)

# فتح الفيديو
video_path = "match_video.mp4"
cap = cv2.VideoCapture(video_path)

# الحصول على خصائص الفيديو
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# إعداد فيديو الإخراج
output_path = "match_analysis4.mp4"
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # صيغة MP4
out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

# إنشاء DataFrame لتسجيل الحالات التحكيمية
columns = ["Frame", "Event", "Confidence"]
events_log = pd.DataFrame(columns=columns)

frame_count = 0
event_active = False  # للتحكم في ظهور الحدث
event_duration = 50   # عدد الإطارات التي يستمر فيها الحدث
event_counter = 0     # عداد الإطارات النشطة للحدث

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_count += 1  # عداد الإطارات

    # تحليل الإطار باستخدام YOLO
    results = model(frame)

    event_text = ""  # نص الحدث لعرضه في الفيديو
    show_yellow_card = False  # للتحكم في إظهار الكرت الأصفر

    for r in results:
        for box in r.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            conf = box.conf[0].item()
            cls = int(box.cls[0].item())

            # تصنيف الكائنات المكتشفة
            if cls == 0:  # اللاعب
                label = "Player"
                color = (0, 255, 0)  # أخضر
            elif cls == 1:  # الحكم
                label = "Referee"
                color = (255, 0, 0)  # أزرق
            elif cls == 2:  # الكرة
                label = "Ball"
                color = (0, 0, 255)  # أحمر
            else:
                label = f"Class {cls}"
                color = (255, 255, 0)  # أصفر

            # رسم الصندوق حول الهدف المكتشف
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
            cv2.putText(frame, f"{label} {conf:.2f}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

            # كشف حالة رفع الحكم للكرت الأصفر
            if cls == 1 and conf > 0.7:  # الحكم يحمل كرتًا
                event_text = "Yellow Card!"
                show_yellow_card = True
                events_log.loc[len(events_log)] = [frame_count, event_text, conf]
                event_active = True  # تفعيل الحدث
                event_counter = event_duration  # ضبط المؤقت

    # التحكم في عرض الحدث فقط عند وقوعه
    if event_active and event_counter > 0:
        # رسم شريط الحدث
        cv2.rectangle(frame, (0, 0), (frame_width, 50), (0, 0, 0), -1)  
        cv2.putText(frame, event_text, (20, 35), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

        # إدراج صورة الكرت الأصفر في أعلى الزاوية اليمنى
        if show_yellow_card:
            card_h, card_w, _ = yellow_card.shape
            frame[10:10+card_h, frame_width-10-card_w:frame_width-10] = yellow_card

        event_counter -= 1  # تقليل العداد
    else:
        event_active = False  # إيقاف الحدث بعد انتهاء المدة

    # حفظ الإطار في الفيديو الجديد
    out.write(frame)

    # عرض الفيديو أثناء التشغيل
    cv2.imshow("Match Analysis", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
out.release()
cv2.destroyAllWindows()

# حفظ التقرير في CSV
events_log.to_csv("match_analysis4.csv", index=False)
print("\n📊 **التقرير النهائي**")
print(events_log)

# عرض إحصائيات التحليل
summary = events_log["Event"].value_counts()
print("\n🔍 **إحصائيات التحكيم**")
print(summary)

print(f"\n✅ تم حفظ الفيديو التحليلي: {output_path3}")


0: 384x640 7 persons, 334.8ms
Speed: 24.9ms preprocess, 334.8ms inference, 3.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 persons, 319.9ms
Speed: 5.4ms preprocess, 319.9ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 persons, 173.6ms
Speed: 4.5ms preprocess, 173.6ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 9 persons, 264.3ms
Speed: 4.9ms preprocess, 264.3ms inference, 2.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 persons, 1 sports ball, 239.3ms
Speed: 28.5ms preprocess, 239.3ms inference, 1.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 persons, 1 sports ball, 168.9ms
Speed: 6.6ms preprocess, 168.9ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 persons, 1 sports ball, 140.2ms
Speed: 7.0ms preprocess, 140.2ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 persons, 1 sports ball, 151.4ms
Spee